In [21]:
import os
import pandas as pd
import csv
import json
import time

from datetime import datetime
from retrying import retry
from retrying import RetryError
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser


DEVELOPER_KEY= os.environ.get('API_CODE')

# DEVELOPER_KEY=[]
# for x in range(2,9):
#     if (x==2):
#         DEVELOPER_KEY.append(os.environ.get('API_CODE'))
#         DEVELOPER_KEY.append(os.environ.get('API_CODE2'))
#     else:
#         DEVELOPER_KEY.append(os.environ.get('API_CODE'+str(x)))
                             
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
# service= build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY[1])
service= build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)

In [22]:
def is_503_error(exception):
    is503=False
    if (isinstance(exception, HttpError)):
        if (exception.resp.status==503):
            is503=True
    return (is503)

In [23]:
@retry(stop_max_attempt_number=7,retry_on_exception=is_503_error, 
       wrap_exception=True,wait_random_min=1000, wait_random_max=2000)
def video_data(video_id):
    videoData=service.videos().list(
        part="id,snippet,statistics,contentDetails,topicDetails",
        id=video_id
    ).execute()
    
    return videoData

In [24]:
@retry(stop_max_attempt_number=7,retry_on_exception=is_503_error, 
       wrap_exception=True,wait_random_min=1000, wait_random_max=2000)
def video_date_keyword(searchKey,pageToken,dt,dt2):
    search_list=service.search().list(
        q=searchKey,
        part="id,snippet",
        maxResults=50,
        order="relevance",
#         order="viewCount",
        type="video",
        publishedAfter=dt,
        publishedBefore=dt2,
        pageToken=pageToken
    ).execute()
    
    return search_list

In [25]:
# get basic channel list with ids (years defined)
def get_videos_list(year,yearParts,day,searchKey,fileName,fileName2,error_fname,prevQuota):
    newQuota=prevQuota
    
    for x in range (1,yearParts+1):
        pageToken=""
        d=datetime(year,int(1+(12/yearParts)*(x-1)),1,0,0).isoformat()+'Z'
        d2=datetime(year,int((12/yearParts)*x),day[x-1],0,0).isoformat()+'Z'
       
        while((pageToken!='CLYHEAA')&(pageToken!='error')):
            videoList=video_date_keyword(searchKey,pageToken,d,d2)
            newQuota+=100
            
            #save list and channels if list is not empty
            if (videoList.get("items",[])!=[]):
                
                #save json dict Channel List 50 items
                with open(fileName,'a')as fp1:
                    out1=json.dumps(videoList)
                    fp1.write(out1 + '\n')
            
                #save the data about the channels in the list
                for video in videoList.get("items", []):
                    try:
                        videoData=video_data(video["id"]["videoId"])
                    except RetryError as e:
                        with open(error_fname, 'a') as f:
                            f.write("max unsuccessful attempts reached"+" id:"+video["id"]["videoId"]+'\n')
                        continue
                    newQuota+=15
                    with open(fileName2,'a')as fp2:
                        out2=json.dumps(videoData)
                        fp2.write(out2 + '\n')
                    time.sleep(1)
            time.sleep(1)
            
            try:
                pageToken=videoList['nextPageToken']
            except KeyError as e:
                pageToken='error'
            
        time.sleep(1)
        
    return newQuota

In [ ]:
# get trimestral info for 10 years of channels related to search keys
yearParts=3
day=[30,31,31]

# read files with the necessary keywords
keys_file= pd.read_csv('keywords.csv')
keys_file['KEYWORD']=keys_file.KEYWORD.apply(lambda x: x.lower())
searchKeys=keys_file.KEYWORD.values

data_videos='/media/aruiz/data/superheroes_data/'
fileName=os.path.join(data_videos,'all_superheroes_list.json')
fileName2=os.path.join(data_videos,'all_superheroes_data.json')
fError=os.path.join(data_videos,'errors.txt')

prevQuota=0
with open('name.txt', 'w') as f:
    for i in range (0, len(searchKeys)):
        for year in range (2009,2019):
            quota=get_videos_list(year,yearParts,day,searchKeys[i],fileName,fileName2,fError, prevQuota)
            prevQuota=quota
            
            f.write(str(i)+'\n') # 3
            f.write(str(year)+'\n') # 2010
            
#             if (quota>=800000):
#                 quota=0
#                 key+=1
#                 if (key==8):
#                     key=0                        
#                 service= build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY[key])
#                 continue
            if (quota>=9500):
                break